# Dementia Baseline Classification Model

This model features a standard (non-contrastive) classification model to be used as a baseline metric.

In [28]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers, metrics
from tensorflow.keras import Input, Model, layers, callbacks, regularizers
from jarvis.train import custom, datasets, params
from jarvis.train.client import Client
from jarvis.utils.general import gpus, overload, tools as jtools

In [29]:
def prepare_client(paths, p):

    client = Client(CLIENT_TRAINING if os.path.exists(CLIENT_TRAINING) else CLIENT_TEMPLATE, configs={
        'batch': {
            'size': p['batch_size'],
            'fold': p['fold']}})

    return client

def load_existing(model, p):

    # --- Create output_dir
    os.makedirs(p['output_dir'], exist_ok=True)

    # --- Load existing model if present
    if os.path.exists(MODEL_NAME):
        print('Loading existing model weights: {}'.format(MODEL_NAME))
        model.load_weights(MODEL_NAME)

    return model

In [30]:
def prepare_model(inputs):

    # --- Define kwargs dictionary
    kwargs = {
        'kernel_size': (3, 3, 3),
        'padding': 'same',
        'kernel_initializer': 'he_uniform'}

    # --- Define lambda functions
    conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
    norm = lambda x : layers.BatchNormalization()(x)
    elu = lambda x : layers.ELU()(x)
    tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)
    
    # --- Define stride-1, stride-2 blocks
    conv1 = lambda filters, x : norm(elu(conv(x, filters, strides=1)))
    conv2 = lambda filters, x : norm(elu(conv(x, filters, strides=(2, 2, 2))))
    tran2 = lambda filters, x : norm(elu(tran(x, filters, strides=(2, 2, 2))))

    # --- Extract alpha value
    a = p['alpha']
    
    # --- Define contracting layers
    l1 = conv1(int(a*8), inputs['dat'])
    l2 = conv1(int(a*16), conv2(int(a*16), l1))
    l3 = conv1(int(a*32), conv2(int(a*32), l2))
    l4 = conv1(int(a*48), conv2(int(a*48), l3))
    l5 = layers.Flatten()(l4)
    l6 = layers.Dense(256, activation='relu')(l5)
    l7 = layers.Dense(64, activation='relu')(l6)
    l8 = layers.Dense(8, activation='relu')(l7)
    
    # --- Create logits
    logits = {}
    logits['lbl'] = layers.Dense(1, activation='softmax', name='lbl')(l8)

    # --- Create model
    model = Model(inputs=inputs, outputs=logits)

    # --- Compile model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=p['LR']), 
        loss={'lbl': losses.BinaryCrossentropy()},
#         metrics={'lbl': metrics.BinaryAccuracy()},
        experimental_run_tf_function=False
    )

    return model

In [31]:
# =======================================================
# PREPARATION 
# =======================================================

# --- Autoselect GPU
gpus.autoselect()

# --- Look up path
paths = jtools.get_paths('adni')

# --- Prepare hyperparams
p = params.load('./hyper.csv', row=0)

# --- Set constants
CLIENT_TEMPLATE = '/home/mmorelan/proj/dementia/yml/client-3d-raw.yml'
CLIENT_TRAINING = '{}/client.yml'.format(p['output_dir'])
MODEL_NAME = '{}/model.hdf5'.format(p['output_dir'])

# --- Prepare client
client = prepare_client(paths, p)
# client.load_data_in_memory()
gen_train, gen_valid = client.create_generators()

# --- Prepare model
model = prepare_model(client.get_inputs(Input))
model = load_existing(model, p)

[ 2021-07-29 16:18:36 ] CUDA_VISIBLE_DEVICES already manually set to: -1       


In [27]:
# =======================================================
# TRAINING LOOP 
# =======================================================

# --- Assume a 400:100 ratio of train:valid
steps_per_epoch = 100
validation_freq = 1

# --- Determine total loop iterations needed
epochs = int(p['iterations'] / steps_per_epoch)

# --- Prepare Tensorboard 
log_dir = '{}/jmodels/logdirs/{}'.format(
    os.path.dirname(p['output_dir']),
    os.path.basename(p['output_dir']))

# --- Train
model.fit(
    x=gen_train, 
    epochs=epochs,
    steps_per_epoch=steps_per_epoch, 
    validation_data=gen_valid,
    validation_steps=steps_per_epoch,
    validation_freq=validation_freq
)

# --- Save model
model.save(MODEL_NAME)

# --- Save client
client.to_yml(CLIENT_TRAINING)

Epoch 1/2


InvalidArgumentError:  data[0].shape = [5] does not start with indices[0].shape = [2]
	 [[node training_4/Adam/gradients/gradients/loss_5/lbl_loss/Mean_grad/DynamicStitch (defined at <ipython-input-27-cd7a34c665bd>:24) ]] [Op:__inference_keras_scratch_graph_22164]

Function call stack:
keras_scratch_graph
